<a href="https://colab.research.google.com/github/adrnabrm/beachhacks-spring25/blob/main/finetuning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
# Installing unsloth
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
# LoRA Configuration
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
alpaca_prompt= """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{ingredients}

### Output:
{recipe}"""

EOS_TOKEN = tokenizer.eos_token

In [7]:
import json
from datasets import Dataset

# Loads the dataset as a Dataset object (using a list of dictionaries, for example)
with open('dataset.json', 'r') as f:
    try:
        data = json.load(f)
        print("JSON is valid")
    except json.JSONDecodeError as e:
        print(f"Error at line {e.lineno}, column {e.colno}: {e.msg}")

# Converts it to a Hugging Face Dataset
dataset = Dataset.from_list(data)

# Define the formatting function
def formatting_prompts_func(examples):
    instructions = examples["system"]
    ingredients = examples["input"]
    recipe = examples["output"]
    texts = []
    for instruction, ingredients, recipe in zip(instructions, ingredients, recipe):
        text = alpaca_prompt.format(instruction=instruction, ingredients=ingredients, recipe=recipe) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Output the formatted dataset
print(dataset[0]['system'])  # To print the first formatted entry


JSON is valid


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

You are an AI chef. Given a list of ingredients, generate a recipe in the following JSON format that will always have a title, ingredients WITH specified measurements, and detailed instructions:
  ```json
{
  "title": "Recipe title",
  "ingredients": ["ingredient1", "ingredient2", ...],
  "directions": ["step1", "step2", ...]
}



In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 250,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML:   0%|          | 0/100000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/100000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,0.794100
2,0.787900
3,0.782400
4,0.820000
5,0.770900
6,0.878600
7,0.750500
8,0.877800
9,0.878500
10,0.803300


In [28]:
instruction = """You are an AI chef. Given a list of ingredients, generate a tasty recipe in the following JSON format that will always have a title, ingredients WITH specified measurements, and detailed instructions:
  ```json
{
  "title": "Recipe title",
  "ingredients": ["ingredient1", "ingredient2", ...],
  "directions": ["step1", "step2", ...]
}"""

ingredients = "Pasta, Chicken breast, Olive oil, Garlic, Lemons, Fresh thyme, Red onion, Cherry tomatoes, Parmesan cheese, Spinach, Balsamic vinegar"
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction=instruction, # instruction
        ingredients=ingredients, # input
        recipe="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 750)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction: 
You are an AI chef. Given a list of ingredients, generate a tasty recipe in the following JSON format that will always have a title, ingredients WITH specified measurements, and detailed instructions:
  ```json
{
  "title": "Recipe title",
  "ingredients": ["ingredient1", "ingredient2",...],
  "directions": ["step1", "step2",...]
}

### Input: 
Pasta, Chicken breast, Olive oil, Garlic, Lemons, Fresh thyme, Red onion, Cherry tomatoes, Parmesan cheese, Spinach, Balsamic vinegar

### Output: 
{'directions': ['Cook the pasta according to the package directions.', 'Meanwhile, heat the olive oil in a large skillet over medium-high heat.', 'Add the chicken and cook until browned, about 5 minutes.', 'Add the garlic and cook for 1 minute.', 'Add the lemon juice, thyme and red onion and cook for 2 minutes

In [29]:
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/tokenizer.json')